# Automatize the generation of an Environment for Grid2op

This notebook will automatize processes to generate an environment for the IEEE 118 grid using *grid2op*. It takes as an input a pre-processed cased and it will generate the respective environment to be used with grid2op.

`case_full_grid_ieee_118 --> | run all notebook | --> subnet ENV grid2op`

The input folder should contain the follwing:
```
|---case_full_grid_ieee_118
    |--- config.py
    |--- grid.json
    |--- grid_layout.json
    |--- loads_charac.csv
    |--- prods_chatac.csv
    |--- params.json
    |--- params_opf.json
```

**Remainder** <br>
**++ ++ ++ +** <br>
 - **It only works for now if you have generated the subgrid case** for grid2op (otherwise go through notebook subgrid_design.ipynb) and preferably for region R2 (not tested for the 2 others

 - **Make sure to have the right slack ids**

 - You need to have **a virtualEnv of Chronix2Grid installed** in Chronix2Grid_folder

 - If you change AWS machine type, you might need to **recompile lightSim2Grid**
 
 - Create virtualenv for Chronix2grid

## Structure and levels:
```
|---main_root
    |---Chronix2grid
    |   |---input_data
    |   |   |---generation
    |   |       |---case_runner_grid (full ieee 118 full grid)
    |   |       |---case_subgrid
    |   |--output_data
    |---Grid2Op_Environment
        |---case_subgrid
    
```

## Processes:

 - Parameters Configuration:
   - number of scenarios
   - number of cores
   - chronix version (for data)
   - subgrid to generate ENV {1, 2, 3}
   - home directory or main_root
   - Slack gen
 - Pre-process input case changing Pmax and Ramps
 
 1) **Chronics Generation For Full IEEE 118** <br>
   *(Run Economic Dispatch using Chronix2grid)* <br>
 2) **Run the runner with the do nothing agent** <br>
   *(Run Grid2op runner from Economic Dispatch results)* <br>
 3) **Correct the slack bus chronic with it** <br>
   *(Run sub-process to compensate slack gen)* <br>
 4) **Create the subgrid chronics** <br>
   *(Using Economic dispatch rel and runner agent, only* <br>
   *specified subnet chronix are created)* <br>
 4) **Run runner in subgrid chronics** <br>
   *(Run Grid2op runner on subgrid chronics)* <br>
 5) **Correct the slack bus chronic** <br>
   *(Correct slack in subgrid chronic)*

## Import libraries

In [ ]:
import os
from pprint import pprint
import pandas as pd
import numpy as np
import subprocess
import shutil
import psutil
from glob import glob
from chronix2grid.main import generate_mp_core

## Parameter Configuration

**--> specify Home directory**

In [ ]:
Chronix2Grid_directory='/Users/antoinemarot/ChroniX2Grid'
#/Users/camiloromero/Rte/"#/home/ubuntu/
# home_directory = '/Users/camiloromero/Rte/'
#home_directory = '/home/ubuntu/'
home_directory = '/Users/antoinemarot/ChroniX2Grid'

In [ ]:
if not os.path.exists(Chronix2Grid_directory):
    print('WARNING: you need to change your home directory '+home_directory+' which does not exist')
    raise

## Input variables

In [ ]:
notebook_config = {
    'full_case_name': 'case118_l2rpn_icaps_2x',
    'subgrid_case_name': 'ieee118_R2subgrid_icaps_2x',
    'num_scenarios': 3,
    'num_cores': 3,#psutil.cpu_count(logical=True),
    'agent_rel_version': '1x',
    'region': 2, 
    'mode_test': True,
    'withseeds': True,
    'path_read_seeds': os.path.join(Chronix2Grid_directory,"tests/data/input/generation","seeds_chronix2grid_118ieee.json"),
    'chronix_env_name': 'venv_chronix2grid',
    'ref_chronix': os.path.join(Chronix2Grid_directory,"input_data/generation","case118_l2rpn_icaps_2x")
}

**--> Replace the number of cores and scenarios**

In [ ]:
# Replace variables to initialize automated process in notebook
n_scenarios = notebook_config['num_scenarios'] #number of scenarios to generate for each month
nb_core = notebook_config['num_cores'] # number of cores on which to run the computation

**--> Replace the name of your virtual env for chronix2grid**

In [ ]:
venv_chronix2grid_name = notebook_config['chronix_env_name']

**--> Replace version data, subregion to generate**

In [ ]:
ChronixVersion = notebook_config['agent_rel_version'] #the version of the iteration we are doing for generating our environment
subgrid_region = notebook_config['region']  #choose subgrid region betwen 1,2,3 for IEEE 118 subgrids

#######
ModeTest = notebook_config['mode_test'] #if Test notebook, this should be true, and everything will be run on 2 chronics over 2 months only to also test for multiprocess

#######
ModeRunnerWithSeeds = notebook_config['withseeds'] # Will generate Chronix with specific seed according the path specified.

**--> specify paths** <br>

In [ ]:
'''Specify Chronix2grid paths'''
Chronix2Grid_folder=Chronix2Grid_directory#os.path.join(Chronix2Grid_directory, "ChroniX2Grid")
Input_folder=os.path.join(Chronix2Grid_folder, "input_data") #chronix2grid input folder with necesaary patterns and grid case
Ouput_folder=os.path.join(Chronix2Grid_folder, "tests/data/output") #output folder for chronics

'''Specify Grid2Op_EnvironmentDesign path'''
RunnerScriptFolder=os.path.join(Chronix2Grid_directory, "getting_started","scripts")

'''Specify full_ieee_118_grid and subgrid NAMES'''
case_runner_grid = notebook_config['full_case_name']
case_subgrid = notebook_config['subgrid_case_name']

'''Specify output folder'''
Ouput_folder_do_nothing=os.path.join(Chronix2Grid_folder, "agent_results" + "_" + ChronixVersion) #output folder when running the runner on chronics

'''Specify path to read seeds'''
seed_read_path = notebook_config['path_read_seeds']

In [ ]:
gridPath=os.path.join(Input_folder, 'generation', case_runner_grid)
if not os.path.exists(gridPath):
    print('WARNING: the grid folder '+gridPath+' does not exist, create it')
    raise

In [ ]:
venv_chronix2grid_path=os.path.join(Chronix2Grid_folder, venv_chronix2grid_name)
if not os.path.exists(venv_chronix2grid_path):
    print('WARNING: the chronix2grid virtual env folder '+venv_chronix2grid_path+' does not exist, create it')
    raise

In [ ]:
# subgridPath=os.path.join(RunnerScriptFolder,case_subgrid)
# if not os.path.exists(subgridPath):
#     print('WARNING: the subgrid folder '+subgridPath+' does not exist, be sure to locate your subgrid folder there.' \
#           'If you have not yet created it, run the subgrid_design.ipynb first')
#     raise

The Process is:
    
    Preliminaries: make sure you have the subgrid case created + create a modified case for the slack to run chroniX2Grid 

    1) Chronics Generation For Full IEEE 118
        - move all chronics in a same folder 
        - duplicate the chronics folder to keep one origal and one that we will modify then
    2) Run the runner with the do nothing agent
        - we want to assess the residual losses that we did not account for in the chronix generation process
    3) Correct the slack bus chronic with it
        - so that no more compensation exists when we will run the chronix again, to avoid compensation making losse management negligeable to the amount of compensation
    4) Create the subgrid chronics
    5) Run the runner with the do nothing agent
        - so that we are all set to analyze the environement and calibrate the thermal limits
        
    6) Last check to see that the slack does not need to compensate and be corrected anymore
     

## 1) Preliminary - create a grid case with lower Pmax and Ramps if needed

<div class="alert alert-block alert-success">
This section may allow you to modify any input file that helps to create an environment in Grid2op and consequentently will affect the Economic Dispatch in the next process.
</div>

<div class="alert alert-block alert-warning">
<b>WARNING:</b> "This can directly be done now in params_opf.json"
</div>

This is the case that will be used for Chronix2Grid generation.

This will allow for slack correction later on the real case

In [ ]:
Input_folder_case=os.path.join(Input_folder,"generation",case_runner_grid)
case_chronix2grid_folder=os.path.join(Input_folder,"generation",case_runner_grid)

In [ ]:
prod_caract_df=pd.read_csv(os.path.join(case_chronix2grid_folder,'prods_charac.csv'))
prod_caract_df.head()

<div class="alert alert-block alert-warning">
<b>WARNING:</b> Make sure that params_opf.json has those parameters: 'loss_grid2op_simulation': True, 'idxSlack': .., 'nameSlack': ..,   "hydro_ramp_reduction_factor":.., "slack_p_max_reduction":.., "slack_ramp_max_reduction":..,
    
    loss_grid2op_simulation will be used to adjust for the losses aposteriori
</div>

change slack and hydro ramps and pmax artificially within dedicated json parameters for better thermal dispatch

In [ ]:
import json 
  
# Opening JSON file 
filePath=os.path.join(case_chronix2grid_folder,'params_opf.json')
f = open(filePath,) 
  
# returns JSON object as  
# a dictionary 
params_json = json.load(f) 
print(params_json)

In [ ]:
# Change losses compensation (this should be below the minimal ratio of losses we will 
#see after running Grid2op on the chronics) 
#Otherwise it will lead to negative slack production
#params_json['reactive_comp'] = 0.5
#params_json['losses_pct'] = 0.4

In [ ]:
#possible changes
#for full grid
#idxSlack=37
#nameSlack='gen_68_37'

#for subgrid
#idxSlack_subgrid=21
#nameSlack_subgrid='gen_35_21'#it has changed name in the subgrid

#PmaxReduction=150
#RampReduction=6
#RampHydroReductionFactor=2

with open(filePath, 'w') as f:
    json.dump(params_json, f)

## 2) Chronics Generation For Full IEEE 118

<div class="alert alert-block alert-success">
Run Economic dispatch using Chronix2Grid.
</div>

It will create two folders:

```
|---main_root
    |---Chronix2grid
        |---{#scenarios}_years_chronics_{version} (1)
        |---{#scenarios}_years_chronics_{version}_SlackCorrection (2)
```

Dir (1) will contain the results of the Economic Dispatch <br>
Dir (2) will be created temporarily with the same result as dir (1)

<code style="background:yellow;color:black">
Copy the full grid case in the output folder to create a full working environment in the end

In [ ]:


orginalGridFolder=os.path.join(Input_folder,'generation',case_runner_grid)
runnerCaseFolder=os.path.join(Ouput_folder,'generation')
#workingGridFolder=os.path.join(RunnerScriptFolder,case_runner_grid)

if os.path.exists(runnerCaseFolder):
    print('delete folder before creating case in:'+runnerCaseFolder)
    !rm -rf $workingGridFolder
!cp -r $orginalGridFolder $runnerCaseFolder #to copy the content of a into b do "cp -r a/. b


In [ ]:
months=["january","february","march","april","may","june",
        "july", "august", "september", "october", "november", "december"]
#months=["november","december"]

start_dates=["2012-01-01", "2012-02-01", "2012-03-01", "2012-04-01", "2012-05-01", "2012-06-01",
             "2012-07-01", "2012-08-01", "2012-09-01", "2012-10-01", "2012-11-01", "2012-12-01"]
#start_dates=["2012-11-01","2012-12-01"]

#os.chdir("ChroniX2Grid/chronix2grid")
os.chdir(Chronix2Grid_folder)
#!source venv_chronix2grid/bin/activate 

#### Read seed form path

In [ ]:
os.getcwd()

In [ ]:
seed = {}
if ModeRunnerWithSeeds:  
    with open(seed_read_path) as f:
        seed = json.load(f)
#if ModeRunnerWithSeeds and os.path.exists(seed_read_path):  
#    for file in glob(os.path.join(seed_read_path, '*.json')):
#        with open(file) as f:
#            seed_file = json.load(f)
#            month = os.path.basename(file).split('_')[0]
#            # Fill seed
#            seed[month] = {'loads': seed_file['loads'],
#                           'renewables': seed_file['renewables'],
#                           'dispatch': seed_file['dispatch'],
#                          }
#        
#pprint(seed, width=1)

check that loss asjustement with grid2Op is activated and check that lightsim2grid is installed for fast computation

In [ ]:
if not params_json['loss_grid2op_simulation']:
    print('activate loss adjustement in params_opf.json to generate proper chronics to be used with grid2op')

In [ ]:
try:
    import lightsim2grid
except ImportError as e:
    print('lightsim2grid is not installed. You should install it for faster loss adjustement computation')  # module doesn't exist, deal with it.


<div class="alert alert-block alert-info">
    Execute command line to run economic dispatch
</div>

<div class="alert alert-block alert-warning">
<b>WARNING:</b> Chronix2grid virtualenv will be activated
</div>

Generate 4 weeks chronics for every month in the Chronix2grid virtual env

In [ ]:
if ModeTest:
    print('we are in testing mode for the process of this notebook')
    months=months[0:2]
    start_dates=start_dates[0:2]
    n_scenarios=2 #number of scenarios to generate for each month
    if(nb_core>4):
        nb_core=4
    
else:
    print('we are in production mode for the process of this notebook, generating lots of chronics')
    
for i in range(len(months)):
    month=months[i]
    print(month)
    start_date=start_dates[i]

    #define your chronix2grid command line with your arguments
    cli_activateVirtualEnv='. '+venv_chronix2grid_path+'/bin/activate'
    
    if ModeRunnerWithSeeds:
        cli_chronix2grid="chronix2grid \
                         --mode LRT --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks 4 --case {} --n_scenarios {} --nb_core {} --scenario_name {} --start-date {} \
                         --seed-for-loads {} --seed-for-res {} --seed-for-dispatch {}".format(
                         Ouput_folder, Input_folder, case_runner_grid, n_scenarios, nb_core, month, start_date, \
                         seed[month]['loads'], seed[month]['renewables'], seed[month]['dispatch'])
        generate_mp_core(case_runner_grid, start_date, 4, 4, n_scenarios, 'LRT',
                     Input_folder, Ouput_folder, str(month),
                     seed[month]['loads'], seed[month]['renewables'], seed[month]['dispatch'], nb_core, ignore_warnings = True)
        
        
    else:
        cli_chronix2grid="chronix2grid \
                         --mode LRT --output-folder {} --input-folder {} --ignore-warnings \
                         --weeks 4 --case {} --n_scenarios {} --nb_core {} --scenario_name {} --start-date {}".format(
                         Ouput_folder, Input_folder, case_runner_grid, n_scenarios, nb_core, month, start_date)
        generate_mp_core(case_runner_grid, start_date, 4, 4, n_scenarios, 'LRT',
                     Input_folder, Ouput_folder, str(month),
                     None, None, None, nb_core, ignore_warnings = True)
        
    #call a subprocess to load your virtual env and excute your cli with it
#     subprocess.call('. venv_chronix2grid/bin/activate;'+cli_chronix2grid, shell=True)
    #output=subprocess.call(cli_activateVirtualEnv+';' + cli_chronix2grid, shell=True)
    

In [ ]:
cli_chronix2grid

First move grid2op runner agent results over chronics (used to do loss adjustement): 
will be reused in case of subgrid chronics generation below

we move it in OutputFolder

In [ ]:
existing_output_case_folder=os.path.join(Ouput_folder, 'generation', case_runner_grid)
agent_result_folder_name='agent_results'
agent_result_folder_path=os.path.join(existing_output_case_folder,agent_result_folder_name)

new_agent_result_folder_path=os.path.join(Ouput_folder,agent_result_folder_name)

if(os.path.exists(agent_result_folder_path)):
    if(os.path.exists(new_agent_result_folder_path)):
        print('delete folder before creating:'+new_agent_result_folder_path)
        !rm -rf $new_agent_result_folder_path
    shutil.move(agent_result_folder_path, Ouput_folder)



Now move monthly folder chronic files into a single chronics folder expected to build a grid2op environment

In [ ]:
ouput_chronics_folder=os.path.join(existing_output_case_folder,'chronics')
if os.path.exists(ouput_chronics_folder):
    print('delete folder before creating:'+ouput_chronics_folder)
    !rm -rf $ouput_chronics_folder

#get monthly chronics folder
existing_subfolderChronics=next(os.walk(existing_output_case_folder))[1]
existing_subfolderChronics=[os.path.join(existing_output_case_folder,subF) for subF in existing_subfolderChronics]


#create chronics folder to move all scenarios in one folder
os.makedirs(ouput_chronics_folder, exist_ok=True)

for d in existing_subfolderChronics:
    #!ls $d/*
    #!mv $d/* $folderChronics
    for sub_d in next(os.walk(d))[1]:
        print(os.path.join(d,sub_d))
        shutil.move(os.path.join(d,sub_d),ouput_chronics_folder)
    
    file_names = os.listdir(d)
    
    for file_name in file_names:
        if(file_name.endswith('.json')):
            print(os.path.join(d, file_name))
            shutil.move(os.path.join(d, file_name), ouput_chronics_folder)
            
    shutil.rmtree(d)
        

In [ ]:
nFolders=n_scenarios*len(months)

if(len(next(os.walk(ouput_chronics_folder))[1])!=nFolders):
    print("we have a problem in the chronics generation - maybe chronics to grid did not run properly")
    print("do it in the shell to check")
    print("do 1)  . venv_chronix2grid/bin/activate")
    print("this will source chronix2grid")
    print("do 2) ")
    print(cli_chronix2grid)
    raise

# Sanity checks on generated chronics

## check that losses have been adjusted

Check adjusted_loss.csv.bz2 files exist

In [ ]:
existing_svcenarios=next(os.walk(ouput_chronics_folder))[1]

for scenario in existing_svcenarios:
    loss_adjustement_file_path=os.path.join(ouput_chronics_folder,scenario,'adjusted_loss.csv.bz2')
   
    if(os.path.exists(loss_adjustement_file_path)):
        loss_adjusted_df=pd.read_csv(loss_adjustement_file_path)
    else:
        print("WARNING: losses have not been adjusted aposteriori with grid2op in scenario: "+scenario)
        print("Maybe dispatch did not converge or grid2op did not run for loss adjustement ")

check that we don't need to ajdust for losses anymore by rerunning once again grid2op and seeing almost no corrections in productions

In [ ]:
script_path=os.path.join(Chronix2Grid_directory,'getting_started/scripts/check_loss_adjustement.py')
%run $script_path

In [ ]:
scen_id=1
existing_svcenarios.sort()
scenario=existing_svcenarios[scen_id]
print(scenario)
max_iter=1000

check_loss_adjustement(existing_output_case_folder,scenario,scen_id,max_iter,Ouput_folder,params_json)

## check that slack production is between Pmin and Pmax have

In [ ]:
name_slack=params_json['nameSlack']
Pmin_slack=0
Pmax_slack=prod_caract_df[prod_caract_df.name==name_slack].Pmax.values[0]
Pmax_slack

In [ ]:

for scenario in existing_svcenarios:
    prod_p_file_path=os.path.join(ouput_chronics_folder,scenario,'prod_p.csv.bz2')
    #print(scenario)
    if(os.path.exists(prod_p_file_path)):
        prod_p=pd.read_csv(prod_p_file_path,sep=';')
        if(name_slack in prod_p.columns):
            min_prod_slack=prod_p[[name_slack]].min()[0]
            max_prod_slack=prod_p[[name_slack]].max()[0]
            if(min_prod_slack<Pmin_slack):
                print("WARNING: for scenario "+scenario+" the min slack production has negative value "+str(min_prod_slack))
            if(max_prod_slack>Pmax_slack):
                print("WARNING: for scenario "+scenario+" the max slack production has high value "+str(max_prod_slack))
        else:
            print("dispatch did not converge for this scenario: "+ scenario)
            
            

### Seed checker

In [ ]:
#TO BE UPDATED
# Path scenarios to compare with different environments
#path_env1 = ouput_chronics_folder
#path_env2 = notebook_config['ref_chronix']
#
#if ModeRunnerWithSeeds:
#    # Select random scenarios
#    rand_month = np.random.choice(months)
#    rand_n_scenario = np.random.choice(range(n_scenarios))
#    if rand_n_scenario < 10:
#        n_scenario = f'0{rand_n_scenario}'
#    else:
#        n_scenario = str(rand_n_scenario)
#
#    # Create proper scenario name
#    rand_scenario_name = f'Scenario_{rand_month}_{rand_n_scenario}'
#
#    factor_renew = 3
#    for var in ['load', 'wind', 'solar']:
#        chrnx_2 = os.path.join(path_env2, 'chronics')
#        df_env1 = pd.read_csv(os.path.join(path_env1, rand_scenario_name, f'{var}_p.csv.bz2'), sep=';')
#        df_env2 = pd.read_csv(os.path.join(chrnx_2, rand_scenario_name, f'{var}_p.csv.bz2'), sep=';')
#        if var not in ['wind', 'solar']:
#            if (df_env1.values[:, 1:] == df_env2.values[:, 1:]).all():
#                print(f'{var}_p are the same in both envs')
#        else:
#            tolr = 1e-01
#            arry1 = factor_renew * df_env1.values[:, 1:].astype(float)
#            arry2 = df_env2.values[:, 1:].astype(float)
#            if np.allclose(arry1, arry2, tolr):
#                print(f'{var}_p are the same in both envs')

#  3) Create the subgrid chronics - Create_chronics_subnet.py

Move subgrid to dir (2)

In [ ]:
FromSubgridPath=os.path.join(Input_folder, 'generation', case_subgrid)
ToSubgridPath=os.path.join(RunnerScriptFolder)
ToSubgridPath

In [ ]:
FromSubgridPath=os.path.join(Input_folder, 'generation', case_subgrid)
ToSubgridPath=os.path.join(RunnerScriptFolder)

!cp -r $FromSubgridPath $ToSubgridPath #to copy the content of a into b do "cp -r a/. b

check that the subgrid folder exists - or otherwise run the 'subgrid_design' notebook

In [ ]:
subgridPath = os.path.join(RunnerScriptFolder, case_subgrid)
folderChronicsSubgrid=os.path.join(subgridPath, 'chronics')
if os.path.exists(folderChronicsSubgrid):
    print('delete folder before creating')
    !rm -rf $folderChronicsSubgrid

os.makedirs(folderChronicsSubgrid, exist_ok=True)

In [ ]:
!ls -lsastr $folderChronicsSubgrid

In [ ]:
!ls -lsastr $ouput_chronics_folder

Check that do-nothing agent results exist from fullgrid grid2op running computation (when doing loss adjustements) as we will reuse those results to set the values of subgrid interconnections

In [ ]:
if not (os.path.exists(new_agent_result_folder_path)):
    print("WARNING: you need to have run grid2op over your chronics for the full grid before creating subgrid chronics")

<div class="alert alert-block alert-info">
    Execute command line to get chronix for subgrid.
</div>

In [ ]:
cli_create_subgrid_chronics="python "+os.path.join(RunnerScriptFolder,'Create_chronics_subnet.py')+' --subnet_path {} --disptach_dir {} --agent_dir {} --ouput_dir {} --region {} --cores {}'.format(
subgridPath,ouput_chronics_folder,new_agent_result_folder_path,folderChronicsSubgrid,subgrid_region,nb_core) 
cli_create_subgrid_chronics


In [ ]:
!$cli_create_subgrid_chronics

Move new subgrid folder in output folder

In [ ]:
ouput_folder_subgrid_path=os.path.join(Ouput_folder,'generation')
new_folder_subgrid_path=os.path.join(ouput_folder_subgrid_path,case_subgrid)

if(os.path.exists(new_folder_subgrid_path)):
    print('delete folder before creating:'+new_folder_subgrid_path)
    !rm -rf $new_folder_subgrid_path

shutil.move(subgridPath,ouput_folder_subgrid_path)
print("you can find your new subgrid environement "+str(case_subgrid)+" in folder: "+ouput_folder_subgrid_path)

In [ ]:
subgridPath

In [ ]:
#output=subprocess.check_output(cli_create_subgrid_chronics,shell=True)

In [ ]:
#print(output.decode('utf-8'))

# Sanity checks on generated chronics

In [ ]:
scen_id=1
existing_svcenarios.sort()
scenario=existing_svcenarios[scen_id]
print(scenario)
max_iter=1000

#change slack info for subgrid in params.json
params_json_subgrid=params_json
params_json_subgrid['idxSlack']=21
params_json_subgrid['nameSlack']='gen_35_21'


check_loss_adjustement(os.path.join(ouput_folder_subgrid_path,case_subgrid),scenario,scen_id,
                       max_iter,Ouput_folder,case_subgrid,params_json_subgrid)
